In [ ]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = 'https://www.rekrute.com/offres-emploi-maroc.html'

In [ ]:
response = requests.get(url)
soup1 = BeautifulSoup(response.text, 'html.parser')
cards = soup1.find_all('li', 'post-id')

Prototype the model with a single record

In [ ]:
card = cards[0] 

In [ ]:
job_title = card.h2.a.text.strip()

In [ ]:
job_url = card.h2.a
job_url = 'https://www.rekrute.com' + job_url.get('href')

In [ ]:
response = requests.get(job_url)
soup2 = BeautifulSoup(response.text, 'html.parser')
cards2 = soup2.find_all('div', 'col-md-10 col-sm-12 col-xs-12')

In [ ]:
job_infos = cards2[0].find_all('li')

In [ ]:
job_loc = job_infos[1].text.strip()

In [ ]:
study_level = job_infos[2].text.strip()

In [ ]:
all_skills = cards2[0].find_all('span','tagSkills')

In [ ]:
skills = ''
for i in range(len(all_skills)):
  skills = all_skills[i].text.strip() + ', ' + skills

In [ ]:
cards3 = soup1.find_all('div','col-md-12 blc')

In [ ]:
business = cards3[1].text.strip()
job = cards3[2].text.strip()
profile = cards3[3].text.strip()
adress = cards3[4].text.strip()

In [ ]:
company = card.a.img.get('title')

In [ ]:
infos = card.find_all('div', 'info')

In [ ]:
# company_info = infos[0].span.text.strip()

In [ ]:
# job_tasks = infos[1].span.text.strip()

In [ ]:
post_date = card.em.span.text

In [ ]:
sub_infos = infos[2].find_all('li')

In [ ]:
sectors = sub_infos[0].text.strip()

In [ ]:
functions = sub_infos[1].find_all('a')
function = functions[0].text.strip() + functions[1].text.strip()

In [ ]:
experience = sub_infos[2].a.text.strip()

In [ ]:
# study_level = sub_infos[3].a.text.strip()

In [ ]:
contract_type = sub_infos[4].a.text.strip()

In [ ]:
record = (job_title, company, job_loc, business, job, profile, adress, post_date, skills, sectors, function, experience, study_level, contract_type)

In [ ]:
record

('Responsable Ressources Humaines Anglophone  (H/F) | Casablanca (Maroc)',
 'GO RH Maroc',
 'Casablanca et région                                     - Maroc',
 "Entreprise :\n\n\tNous sommes un cabinet spécialisé en ressources humaines pour les start-ups et les entreprises, nous accompagnons nos clients en Recrutement, Executive search, RPO et Conseil au Maroc et à l’international.\n\n\n\tNos consultants construisent, avec les décideurs et leurs collaborateurs, des projets entièrement adaptés à leurs problématiques et aux contextes.\n\n\n\tNous savons, de par nos différentes expertises métiers et marchés, que chaque environnement et chaque culture sont confrontés à des leviers stratégiques et opérationnels spécifiques qui varient fréquemment avec les cycles d’activité, et que les salariés, en particulier les cadres et les dirigeants, doivent régulièrement en être porteurs.\n\n\tnous recrutons pour notre client, un mastodon du BTP à l'échelle Internationale , un Responsable Ressources 

Generalize the model with a function

In [ ]:
def get_record(card):
    """Extract job data from a single record"""
    job_url = card.h2.a
    job_url = 'https://www.rekrute.com' + job_url.get('href')

    response = requests.get(job_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards2 = soup.find_all('div', 'col-md-10 col-sm-12 col-xs-12')    
    job_title = card.h2.a.text.strip()
    try:
      company = card.a.img.get('title')
    except:
      company = ' '
    if len(cards2)==0:
        return '' 
           
    job_loc = job_infos[1].text.strip()
    study_level = job_infos[2].text.strip()

    all_skills = cards2[0].find_all('span','tagSkills')

    skills = ''
    for i in range(len(all_skills)):
      skills = all_skills[i].text.strip() + ', ' + skills

    cards3 = soup.find_all('div','col-md-12 blc')
    business = cards3[1].text.strip()
    job = cards3[2].text.strip()
    profile = cards3[3].text.strip()
    adress = cards3[4].text.strip()

    infos = card.find_all('div', 'info')
    sub_infos = infos[2].find_all('li')
    sectors = sub_infos[0].text.strip()
    functions = sub_infos[1].find_all('a')
    function = functions[0].text.strip() + functions[1].text.strip()
    
    post_date = card.em.span.text

    experience = sub_infos[2].a.text.strip()

    contract_type = sub_infos[4].a.text.strip()

    record = (job_title, company, job_loc, business, job, profile, adress, post_date, skills, sectors, function, experience, study_level, contract_type)
    return record

In [ ]:
get_record(card)

('Responsable Ressources Humaines Anglophone  (H/F) | Casablanca (Maroc)',
 'GO RH Maroc',
 'Casablanca et région                                     - Maroc',
 "Entreprise :\n\n\tNous sommes un cabinet spécialisé en ressources humaines pour les start-ups et les entreprises, nous accompagnons nos clients en Recrutement, Executive search, RPO et Conseil au Maroc et à l’international.\n\n\n\tNos consultants construisent, avec les décideurs et leurs collaborateurs, des projets entièrement adaptés à leurs problématiques et aux contextes.\n\n\n\tNous savons, de par nos différentes expertises métiers et marchés, que chaque environnement et chaque culture sont confrontés à des leviers stratégiques et opérationnels spécifiques qui varient fréquemment avec les cycles d’activité, et que les salariés, en particulier les cadres et les dirigeants, doivent régulièrement en être porteurs.\n\n\tnous recrutons pour notre client, un mastodon du BTP à l'échelle Internationale , un Responsable Ressources 

In [ ]:
records = []

for card in cards:
    record = get_record(card)
    if record == '':
        continue
    records.append(record)

In [ ]:
len(records)

8

Get the next pages

In [ ]:
records = []
while True:
  try:
    url = 'https://www.rekrute.com/' + soup.find('a','next').get('href')
  except AttributeError:
    break
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  cards = soup.find_all('li', 'post-id')
  for card in cards:
    record = get_record(card)
    if record == '':
        continue
    records.append(record)

In [ ]:
len(records)

1309

In [ ]:
rekrute_data = pd.DataFrame(records, columns=['JobTitle', 'CompanyName', 'Location', 'Company', 'JobSummary', 'Profile', 'adress', 'PostDate', 'Skills', 'Sectors', 'Function', 'Experience', 'StudyLevel', 'ContractType'])

In [ ]:
rekrute_data

,JobTitle,CompanyName,Location,Company,JobSummary,Profile,adress,PostDate,Skills,Sectors,Function,Experience,StudyLevel,ContractType
0,Superviseur nettoyage - Rabat | RABAT (Maroc),Université Mohammed VI Polytechnique,Casablanca et région ...,Entreprise :\nMohammed VI Polytechnic Universi...,Poste :\nDans le cadre du développement de ses...,Profil recherché :\nCompétences Très bon nivea...,"Adresse :\nLot 660, Hay Moulay Rachid, Ben Gue...",28/12/2021,"Implication au travail, Besoin d'action, Respe...",Secteur d'activité : Autres services,Logistique / Transport (métiers de),De 3 à 5 ans,Bac +5 et plus,CDI
1,Responsable magasin et PME -Rabat | RABAT (Maroc),Université Mohammed VI Polytechnique,Casablanca et région ...,Entreprise :\nMohammed VI Polytechnic Universi...,Poste :\nDans le cadre du développement de ses...,Profil recherché :\nCompétences Très bon nivea...,"Adresse :\nLot 660, Hay Moulay Rachid, Ben Gue...",28/12/2021,"Implication au travail, Besoin d'action, Respe...",Secteur d'activité : Autres services,Logistique / Transport (métiers de),De 3 à 5 ans,Bac +5 et plus,CDI
2,Gouvernant(e) | RABAT (Maroc),Université Mohammed VI Polytechnique,Casablanca et région ...,Entreprise :\nMohammed VI Polytechnic Universi...,Poste :\nDans le cadre du développement de ses...,Profil recherché :\n Compétences Encadrer de...,"Adresse :\nLot 660, Hay Moulay Rachid, Ben Gue...",28/12/2021,"Implication au travail, Besoin d'action, Respe...",Secteur d'activité : Autres services,Logistique / Transport (métiers de),De 3 à 5 ans,Bac +5 et plus,CDI
3,Management control | RABAT (Maroc),Université Mohammed VI Polytechnique,Casablanca et région ...,Entreprise :\nMohammed VI Polytechnic Universi...,Poste :\nDans le cadre du développement de ses...,Profil recherché :\n Compétences techniques A...,"Adresse :\nLot 660, Hay Moulay Rachid, Ben Gue...",28/12/2021,"Implication au travail, Besoin d'action, Respe...",Secteur d'activité : Autres services,Logistique / Transport (métiers de),De 3 à 5 ans,Bac +5 et plus,CDI
4,Coordinateur Facility Management | BENGUERIR (...,Université Mohammed VI Polytechnique,Casablanca et région ...,Entreprise :\nMohammed VI Polytechnic Universi...,Poste :\nDans le cadre du développement de ses...,Profil recherché :\nAccessible avec Un Bac+5 e...,"Adresse :\nLot 660, Hay Moulay Rachid, Ben Gue...",28/12/2021,"Implication au travail, Besoin d'action, Respe...",Secteur d'activité : Autres services,Logistique / Transport (métiers de),De 3 à 5 ans,Bac +5 et plus,CDI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,Chargé(e) de Recouvrement - Anglophone | Casab...,Maroc Climate and Security (MCS) - Carrier,Casablanca et région ...,Entreprise :\n\n\tC’est en 1902 que le Dr Will...,"Poste :\n\n\tActuellement, on recrute pour le ...",Profil recherché :\n\n\tDe formation :\n\t-Bac...,"Adresse :\nImmeuble LA COMANAV, 7 Boulevard la...",01/11/2021,"Organisation, Besoin de réflexion, Flexibilité...",Secteur d'activité : Autres services,Call Centers (métiers de)Avocat / Juriste / Fi...,De 1 à 3 ans,Bac +5 et plus,CDD
1305,Coordinateur Qualité Produit | AGADIR (Maroc),,Casablanca et région ...,Entreprise :\n\n\tLeader dans notre secteur (E...,Poste :\n\n\n\t\tEffectue les vérifications et...,Profil recherché :\n\n\n\t\tFormation qualifia...,Traits de personnalité souhaités :\n\n\r\n ...,01/11/2021,"Respect des règles, Rationalisme, Travail en é...",Secteur d'activité : Autres services,Production / Qualité / Sécurité / MaintenanceP...,De 1 à 3 ans,Bac +5 et plus,CDI
1306,Animateur HSE | El Jadida (Maroc),Meski Invest - Riva Industries,Casablanca et région ...,Entreprise :\n\n\tRiva industrie complexe sidé...,Poste :\n\n\tNous recrutons des animateurs HSE...,Profil recherché :\n\n\tDe formation HSE av...,"Adresse :\nRésidence Marina Center 1, Angle Bo...",31/10/2021,"Respect des règles, Rationalisme, Ténacité, Be...",Secteur d'activité : Métallurgie / Sidérurgie,Production / Qualité / Sécurité / MaintenanceP...,De 1 à 3 an

In [ ]:
rekrute_data.to_csv('rekrute_data.csv')